# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
price_data = os.getenv("PRICE_DATA")
parquet_file_paths = glob(os.path.join(price_data, '**/*.parquet'), recursive=True)

print(parquet_file_paths)


['../../05_src/data/prices/AAPL/AAPL_2010/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2010/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2017/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2017/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2021/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2021/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2019/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2019/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2018/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2018/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2020/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2020/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2016/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2016/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2011/part.0.parquet', '../../05_src/data/prices/AAPL/AAPL_2011/part.1.parquet', '../../05_src/data/prices/AAPL/AAPL_2002/part.0.parquet', '../../05_src

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_df = dd.read_parquet(parquet_file_paths)

meta = dd_df._meta.copy()

# print(meta)

meta['Close_lag'] = meta['Close']
meta['Adj_Close_lag'] = meta['Adj Close']
meta['returns'] = meta['Close']
meta['hi_lo_range'] = meta['High']

dd_features = dd_df.groupby('Ticker').apply(
    lambda df: df.sort_values('Date').assign(
        Close_lag = lambda x: x['Close'].shift(1),
        Adj_Close_lag = lambda x: x['Adj Close'].shift(1),
        returns = lambda x: (x['Close']/x['Close'].shift(1))-1,
        hi_lo_range = lambda x: x['High'] - x['Low'],
    ),
    meta=meta
)   


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [11]:
# Write your code below.

pd_df = dd_features.compute()
pd_df = pd_df.reset_index(drop=True)

pd_df = pd_df.assign(returns_ma_10=lambda x: x.groupby('Ticker')['returns'].transform(lambda y: y.rolling(10).mean()))

# print(pd_df)

                            Date Ticker   Adj Close       Close        High  \
0      2000-01-03 00:00:00+00:00   JNPR   44.405510   57.333332   58.333332   
1      2000-01-04 00:00:00+00:00   JNPR   42.453083   54.812500   58.833332   
2      2000-01-05 00:00:00+00:00   JNPR   42.130371   54.395832   56.666668   
3      2000-01-06 00:00:00+00:00   JNPR   40.274757   52.000000   56.500000   
4      2000-01-07 00:00:00+00:00   JNPR   41.936749   54.145832   54.166668   
...                          ...    ...         ...         ...         ...   
403451 2025-01-17 00:00:00+00:00   NTAP  122.089996  122.089996  122.750000   
403452 2025-01-21 00:00:00+00:00   NTAP  123.669998  123.669998  124.300003   
403453 2025-01-22 00:00:00+00:00   NTAP  126.050003  126.050003  127.190002   
403454 2025-01-23 00:00:00+00:00   NTAP  126.300003  126.300003  126.300003   
403455 2025-01-24 00:00:00+00:00   NTAP  126.000000  126.000000  126.620003   

               Low        Open     Volume  Year   C

In [12]:
# save pd_df to a csv file
pd_df.to_csv('features.csv', index=False)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
> No, it’s not necessary.
+ Would it have been better to do it in Dask? Why?
> It depends on your data size:
Use Dask for large datasets that don’t fit in memory or need parallel processing.
Use Pandas for small datasets because it’s faster and simpler for in-memory computations.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.